In [327]:
import pandas as pd
puzzle = r"C:\Users\lenovo\Desktop\test4.xlsx"
read_puzzle = pd.read_excel(puzzle,header=None).fillna(0).astype(int).replace(0,"")
padding = pd.DataFrame(index=range(9),columns=range(9),data="")
read_puzzle = pd.concat([read_puzzle,padding])
display(read_puzzle.iloc[:9,:9])
p_dict = {i*9+j: read_puzzle.iloc[i,j] for i in range(9) for j in range(9) if read_puzzle.iloc[i,j] in range(1,10)}

,0,1,2,3,4,5,6,7,8
0,,,,,,,,,
1,,,,,,,,,
2,,,,,,,,,
3,,,,,,,,,
4,,,,,,,,,
5,,,,,,,,,
6,,,,,,,,,
7,,,,,,,,,
8,,,,,,,,,


In [328]:
r_dict = {i: [i*9 + j for j in range(9)] for i in range(9)}
c_dict = {i: [j*9 + i for j in range(9)] for i in range(9)}
sq_dict = {i: [(i//3*3+k)*9+i%3*3+j for j in range(3) for k in range(3)] for i in range(9)}
v_dict = {i: {"r":r_dict[i//9], "c":c_dict[i%9], "sq":[sq_dict[n] for n in sq_dict if i in sq_dict[n]][0]} for i in range(81)}

In [329]:
from collections import defaultdict 
def count_dict (d):
    d2 = defaultdict(int)
    for key in d:
        d2[d[key]] += 1
    return d2

def validate_one (p_dict,positions=None):
    if positions:
        val = {e: p_dict[e] for e in positions if e in p_dict}
    else:
        val = p_dict
    count = count_dict(val)
    for num in count.keys():
        if count[num] > 1:
            return False
    return True

def validate_all (p_dict):
    for d in (r_dict,c_dict,sq_dict):
        for r in d:
            if validate_one(p_dict,d[r]) == False:
                return False
    return True

def validate_three (p_dict, position):
    for r in ("r","c","sq"):
        if validate_one(p_dict,v_dict[position][r]) == False:
            return False
    return True

def apply_one (p_dict,position,num):
    d = p_dict.copy()
    d[position] = num
    return d

def p_try (p_dict):
    num_count = count_dict(p_dict)
    usable = [num for num in range(1,10) if num_count[num] != 9 or num not in num_count]
    empty = [p for p in range(81) if p not in p_dict]
    p_try_dict = {}
    for i in empty:
        for n in usable:
            if validate_three(apply_one(p_dict,i,n),i) == True:
                if i in p_try_dict.keys():
                    p_try_dict[i] += [n]
                else:
                    p_try_dict[i] = [n]
        if i not in p_try_dict.keys():
            p_try_dict[i] = []
    return (p_try_dict)

def subdict (pv,positions):
    return {p: pv[p] for p in positions if p in pv}

def allow_part (smalldict):
    temp = []
    for key in smalldict:
        if smalldict[key] in temp:
            return False
        else:
            temp += [smalldict[key]]
    return True

store = []
def simple_fill (pv):
    global store
    max_results = 3
    if len(store) >= max_results:
        return
    possibilities = p_try(pv)
    if len(possibilities) == 0:
        store.append(pv)
    else:
        length_possibilities = [len(possibilities[i]) for i in possibilities]
        if min(length_possibilities) == 0:
            return None
        elif min(length_possibilities) == 1:
            fill_immediately = {i: possibilities[i][0] for i in possibilities if len(possibilities[i])==1}
            pv_c = pv.copy()
            for i in fill_immediately:
                pv_c[i] = fill_immediately[i]
            simple_fill(pv_c)
        else:
            min_possibility = sorted([(i, possibilities[i]) for i in possibilities], key = lambda x: len(x[1]))
            trials = min_possibility[0]
            for trial in trials[1]:
                pv_c = pv.copy()
                pv_c[trials[0]] = trial
                simple_fill(pv_c)

def dictbacktodf (p_dict):
    pv = p_dict.copy()
    pv = {p:pv[p] if p in pv else 0 for p in range(81)}
    dict_list = [[pv[i*9+j] for j in range(9)] for i in range(9)]
    df = pd.DataFrame(dict_list)
    return df

simple_fill(p_dict)
display(dictbacktodf(store[0]))
print(len(store))
print(validate_all(store[0]))

,0,1,2,3,4,5,6,7,8
0,1,2,3,4,5,6,7,8,9
1,4,5,6,7,8,9,1,2,3
2,7,8,9,1,2,3,4,5,6
3,2,3,1,6,7,4,8,9,5
4,8,7,5,9,1,2,3,6,4
5,6,9,4,5,3,8,2,1,7
6,3,1,7,2,6,5,9,4,8
7,5,4,2,8,9,7,6,3,1
8,9,6,8,3,4,1,5,7,2


3
True


In [137]:
from collections import defaultdict 
def count_dict (d):
    d2 = defaultdict(int)
    for key in d:
        d2[d[key]] += 1
    return d2

def validate_one (p_dict,positions=None):
    if positions:
        val = {e: p_dict[e] for e in positions if e in p_dict}
    else:
        val = p_dict
    count = count_dict(val)
    for num in count.keys():
        if count[num] > 1:
            return False
    return True



def validate_three (p_dict, position):
    for r in ("r","c","sq"):
        if validate_one(p_dict,v_dict[position][r]) == False:
            return False
    return True

def allow_values (p_dict, usable, positions):
    select = {p: p_dict[p] for p in positions if p in p_dict}
    num_count = count_dict(select)
    usable2 = [num for num in usable if num not in num_count]
    p_tryd = {}
    for e in positions:
        for n in usable2:
            if validate_one(apply_one(select,e,n)) == True:
                if e in p_tryd:
                    p_tryd[e] += [n]
                else:
                    p_tryd[e] = [n]
    return(p_tryd)
    

In [139]:
def apply_one (p_dict,position,num):
    d = p_dict.copy()
    d[position] = num
    return d

def p_try (p_dict):
    num_count = count_dict(p_dict)
    usable = [num for num in num_count if num_count[num] != 9]
    empty = [p for p in range(81) if p not in p_dict]
    p_try_dict = {}
    for i in empty:
        for n in usable:
            if validate_three(apply_one(p_dict,i,n),i) == True:
                if i in p_try_dict.keys():
                    p_try_dict[i] += [n]
                else:
                    p_try_dict[i] = [n]
    return (p_try_dict)
print(allow_values(p_dict,usable,sq_dict[3]))

{27: [7, 3], 36: [7, 3], 45: [7, 3], 28: [7, 3], 37: [7, 3], 46: [7, 3], 29: [7, 3], 38: [7, 3], 47: [7, 3]}


In [177]:
%%timeit -n 30000 -r 1
p_dict[29]
#p_list[29]

136 ns ± 0 ns per loop (mean ± std. dev. of 1 run, 30000 loops each)
